# ExperimentRunner Demo: Bulk Experimentation

The `ExperimentRunner` enables systematic benchmarking across multiple parameters. Let's run a small batch experiment.

For transpiling to quantum hardware, we need to initialize backends first. Let's set up IBM Quantum access.

### How to Get IBM Credentials

To transpile to, or use real IBM Quantum hardware:

1. **Sign up** at [IBM Quantum Platform](https://quantum-computing.ibm.com/)
2. **Create an instance CRN** you currently need to setup a Pay-As-Yo-Go account at least
3. **Get your API token** directly from IBM Quantum Platform
4. **Choose a device** like `ibm_brisbane`, `ibm_torino`, etc.

In [1]:
# Import the BackendManager for hardware backends
from sudoku_nisq.backends import BackendManager

# Initialize IBM Quantum backend
# Note: Replace with your actual IBM API token and instance
# For demo purposes, we'll show the initialization code but comment it out

# Modify these lines with your credentials:
# api_token = "your_ibm_api_token_here"
# instance = "crn:v1:bluemix:public:quantum-computing:us-east:a/your_instance_id"

api_token = "MgUA1d64SPwFrqy-C1FnFYUY7lQG4B2F1k0xie5bUcW5"
instance = "crn:v1:bluemix:public:quantum-computing:us-east:a/53bccd1b6f1943a486285adb9d2dfa3f:c8244fef-9afe-428a-8e1a-08a6a03e61e9::"

BackendManager.authenticate_ibm(
    api_token=api_token,
    instance=instance
    )

IBM authentication successful
Found 6 IBM devices available to your account
Available devices: ['ibm_brisbane', 'ibm_fez', 'ibm_sherbrooke', 'ibm_torino', 'ibm_marrakesh', 'ibm_kingston']


['ibm_brisbane',
 'ibm_fez',
 'ibm_sherbrooke',
 'ibm_torino',
 'ibm_marrakesh',
 'ibm_kingston']

In [2]:
# Add backends from available devices

BackendManager.add_ibm_device(device="ibm_torino", alias="ibm_torino")
BackendManager.add_ibm_device(device="ibm_brisbane", alias="ibm_brisbane")

# Check registered backends
available_backends = BackendManager.aliases()
print(f"Registered backends: {available_backends}")

Registered backends: ['ibm_torino', 'ibm_brisbane']


You can also `BackendManager.init_ibm()` method handles both authentication and device registration in one step. Once initialized, you can access the backend anywhere in your code using `BackendManager.get("ibm_brisbane")`.

In [ ]:
# Import the convenience function for batch experiments
from sudoku_nisq import run_experiment_batch

# Run a small batch experiment
# This will test ExactCoverQuantumSolver on the two backends, with 2 optimisation levels,
# with 2 encodings on 2 different puzzle difficulties, with 2 samples each

# Import solver class
from sudoku_nisq.exact_cover_solver import ExactCoverQuantumSolver

# Define the backends to test
backends_to_test = ["ibm_brisbane", "ibm_torino"]

csv_path = run_experiment_batch(
    solvers_config={
        ExactCoverQuantumSolver: ["simple", "pattern"]  # 2 encodings
    },
    backends=backends_to_test,
    opt_levels=[0, 1],          # Test with optimization levels 0 and 1
    puzzle_sizes=[2],        # 4x4 puzzles only
    missing_cells=[2,4],      # 2 to 4 missing cells
    samples_per_combo=2,     # 1 puzzle samples per combination
    canonicalize=True,  # Canonicalize puzzles
    show_progress=True,  # Show real-time progress
    csv_path="demo_experiment.csv",
    cache_base="./.demo_cache",
    cache_transpiled=True,
    progress_interval=4  # Update progress every 4 rows
)

print(f"\nExperiment completed! Results saved to: {csv_path}")

Loaded 6 seen hashes from demo_experiment.csv
Starting batch: 40 total CSV rows
CSV output: demo_experiment.csv
Cache base: .demo_cache
Progress: 4/40 CSV rows (10.0%) - Memory: 368.0 MB
Completed ExactCoverQuantumSolver with encoding 'simple' for puzzle b4175580439197c732c261fcda5d4a30e25e92827f7d89febd72572ad461516e
Progress: 8/40 CSV rows (20.0%) - Memory: 373.8 MB
Completed ExactCoverQuantumSolver with encoding 'pattern' for puzzle b4175580439197c732c261fcda5d4a30e25e92827f7d89febd72572ad461516e
Memory after puzzle cleanup: 377.3 MB
Progress: 12/40 CSV rows (30.0%) - Memory: 383.9 MB
Completed ExactCoverQuantumSolver with encoding 'simple' for puzzle f93b980f17f82265a8c922b15510f9e21cd136773f0770bbb8e2c4dee37c9284
Progress: 16/40 CSV rows (40.0%) - Memory: 390.8 MB
Progress: 20/40 CSV rows (50.0%) - Memory: 400.1 MB
Completed ExactCoverQuantumSolver with encoding 'pattern' for puzzle f93b980f17f82265a8c922b15510f9e21cd136773f0770bbb8e2c4dee37c9284
Memory after puzzle cleanup: 400.1

In [ ]:
# Let's examine the results
import pandas as pd
import os

if os.path.exists("demo_experiment.csv"):
    df = pd.read_csv("demo_experiment.csv")
    
    print(f"Experiment generated {len(df)} CSV rows")
    print(f"Columns: {list(df.columns)}")
    
    print("\nBreakdown by solver and encoding:")
    breakdown = df.groupby(['solver_name', 'encoding']).size()
    print(breakdown)
    
    print("\nSample of circuit resources:")
    resource_cols = ['main_n_qubits', 'main_n_gates', 'main_depth']
    available_cols = [col for col in resource_cols if col in df.columns]
    if available_cols:
        print(df[['solver_name', 'encoding'] + available_cols].head())
    
    print("\nAny errors encountered:")
    if 'error' in df.columns:
        errors = df[df['error'].notna()]
        if len(errors) > 0:
            print(f"Found {len(errors)} errors")
            print(errors[['solver_name', 'encoding', 'error']].head())
        else:
            print("No errors found - all experiments succeeded!")
    
else:
    print("CSV file not found - experiment may not have completed")

## What Just Happened?

The `ExperimentRunner` systematically tested:

- **2 solver encodings** (simple, pattern)
- **2 puzzle difficulties** (3 and 5 missing cells)  
- **2 samples per combination** (for statistical reliability)
- **1 backend** (simulator)

This generated **8 total CSV rows**:
- 4 puzzles × 2 encodings × 1 main circuit row each = 8 rows

Each row contains:
- Puzzle metadata (size, missing cells, hash)
- Solver configuration (name, encoding) 
- Circuit resources (qubits, gates, depth)
- Any errors encountered

## Key Benefits

1. **Systematic**: Tests all parameter combinations
2. **Robust**: Continues even if individual experiments fail
3. **Comprehensive**: Logs everything to CSV for analysis
4. **Memory-safe**: Automatic cleanup prevents memory leaks
5. **Progress tracking**: Real-time updates during long runs